<a href="https://colab.research.google.com/github/Igorgodoi/tccpucminas/blob/master/TCCMINAS_AUTOML_H2O.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
# Autor: Igor Reis de Godoi
# TCC PUCMINAS 2020 - Predição da duração de uma audiência trabalhista
# Quarta Parte (Plus): Criação de Modelos de Machine Learning, utilizando AutoML H2O

In [4]:
from google.colab import drive
import matplotlib.pyplot as plt
%matplotlib inline
%pylab inline
import numpy as np
import pandas as pd
plt.style.use('ggplot')

Populating the interactive namespace from numpy and matplotlib


In [5]:
# Importando o dataset audiencias processada
drive.mount('/content/drive')
df_audiencia = pd.read_csv('/content/drive/My Drive/Colab Notebooks/datasets/audiencia_processada.csv', encoding='CP1252')
# Eliminando coluna municipio, pois é redundante com nome_municipio
df_audiencia=df_audiencia.drop(['municipio','Unnamed: 0','dt_inicio_programada','hora_abertura','hora_encerramento', 'data_audiencia' , 'dia_semana', 'minutos_do_dia'], axis=1)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [6]:
# Ajustando tipagem de colunas
colunas_categoricas = ['uf_municipio', 'nome_municipio', 'tipo_audiencia', 'vara_trabalho', 'classe_processual', 'juiz', 'sala_audiencia', 
                       'atividade_economica', 'assunto', 'autor', 'reu','marcacao_automatica', 'apos_reforma_trabalhista', 'segredo_justica',
                       'justica_gratuita','tutela_liminar','prioridade']
for coluna in colunas_categoricas:
  df_audiencia[coluna] = df_audiencia[coluna].astype('category')

In [7]:
# Dividindo os dados em treino e teste:
from sklearn.model_selection import train_test_split
from sklearn import metrics
X_train, X_test, y_train, y_test = train_test_split(df_audiencia.drop('duracao_audiencia',axis=1),df_audiencia['duracao_audiencia'],test_size=0.3, random_state=0)
df_audiencia_train=pd.concat([X_train,y_train], axis=1)
df_audiencia_test=pd.concat([X_test,y_test], axis=1)

In [8]:
# Instalando bilbioteca h2o
!pip install h2o

In [11]:
import h2o
h2o.init()
hX_train=h2o.H2OFrame(X_train)
hX_test=h2o.H2OFrame(X_test)
hf_audiencia=h2o.H2OFrame(df_audiencia)
hf_audiencia_train=h2o.H2OFrame(df_audiencia_train)
hf_audiencia_test=h2o.H2OFrame(df_audiencia_test)
#hy_train=h2o.H2OFrame(y_train)
#hy_test=h2o.H2OFrame(y_test)


Checking whether there is an H2O instance running at http://localhost:54321 . connected.


H2O_cluster_uptime:,7 mins 29 secs
H2O_cluster_timezone:,Etc/UTC
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.30.1.1
H2O_cluster_version_age:,4 days
H2O_cluster_name:,H2O_from_python_unknownUser_zm3iux
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,3.172 Gb
H2O_cluster_total_cores:,2
H2O_cluster_allowed_cores:,2
H2O_cluster_status:,"locked, healthy"


Parse progress: |█████████████████████████████████████████████████████████| 100%
Parse progress: |█████████████████████████████████████████████████████████| 100%
Parse progress: |█████████████████████████████████████████████████████████| 100%
Parse progress: |█████████████████████████████████████████████████████████| 100%
Parse progress: |█████████████████████████████████████████████████████████| 100%


In [15]:
from h2o.automl import H2OAutoML
y='duracao_audiencia'
aml2 = H2OAutoML(max_runtime_secs = 600, seed = 22, project_name = "PredicaoTempoAudiencia")
aml2.train(y = y, training_frame = hf_audiencia)

AutoML progress: |████████████████████████████████████████████████████████| 100%


In [16]:
aml2.leaderboard.head(50)

model_id,mean_residual_deviance,rmse,mse,mae,rmsle
StackedEnsemble_AllModels_AutoML_20200814_173824,124.633,11.1639,124.633,7.05261,nan
StackedEnsemble_BestOfFamily_AutoML_20200814_173824,125.139,11.1865,125.139,7.08518,nan
XGBoost_grid__1_AutoML_20200814_173824_model_1,126.085,11.2287,126.085,7.13155,nan
XGBoost_grid__1_AutoML_20200814_173824_model_2,128.168,11.3211,128.168,7.08365,nan
GLM_1_AutoML_20200814_173824,135.686,11.6484,135.686,7.57794,0.854139
DeepLearning_1_AutoML_20200814_173824,136.428,11.6803,136.428,7.78823,nan
GBM_grid__1_AutoML_20200814_173824_model_2,142.462,11.9357,142.462,7.99935,0.91626
XGBoost_3_AutoML_20200814_173824,145.639,12.0681,145.639,7.38544,0.842142
XGBoost_1_AutoML_20200814_173824,147.872,12.1603,147.872,7.14149,0.806922
GBM_2_AutoML_20200814_173824,149.115,12.2113,149.115,8.27835,0.9468
